In [1]:
import pandas as pd
import numpy as np
import pickle

from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import entropy

from joblib import Parallel, delayed

In [2]:
# Load data
data = pd.read_csv("vdf.csv")

data.age = pd.factorize(data.age)[0]
data.v = pd.factorize(data.v)[0]
data = data.fillna(0)
print(data)

col_names = ["claw", "dist", "age", "cluster"]
X = np.array(data[col_names])
y = np.array(pd.factorize(data.type)[0])
print(X.shape)
print(y.shape)

       v type  claw        dist  age  cluster
0      0   KC   2.0   337.63801    0        1
1      1   KC   1.0     0.00000    0        1
2      2   KC   1.0  1291.20357    0        1
3      3   KC   1.0   252.63347    0        1
4      4   KC   2.0  1822.29525    0        1
..   ...  ...   ...         ...  ...      ...
208  208   PN   0.0     0.00000   -1        6
209  209   PN   0.0     0.00000   -1        6
210  210   PN   0.0     0.00000   -1        6
211  211   PN   0.0     0.00000   -1        6
212  212   PN   0.0     0.00000   -1        6

[213 rows x 6 columns]
(213, 4)
(213,)


In [3]:
def uf(X, y, n_estimators = 300, max_samples = .4, base = np.exp(1), kappa = 3):
    
    # Build forest with default parameters.
    model = BaggingClassifier(DecisionTreeClassifier(), 
                              n_estimators=n_estimators, 
                              max_samples=max_samples, 
                              bootstrap=False)
    model.fit(X, y)
    n = X.shape[0]
    K = model.n_classes_
    _, y = np.unique(y, return_inverse=True)
    
    cond_entropy = 0
    for tree_idx, tree in enumerate(model):
        # Find the indices of the training set used for partition.
        sampled_indices = model.estimators_samples_[tree_idx]
        unsampled_indices = np.delete(np.arange(0,n), sampled_indices)
        
        # Randomly split the rest into voting and evaluation.
        total_unsampled = len(unsampled_indices)
        np.random.shuffle(unsampled_indices)
        vote_indices = unsampled_indices[:total_unsampled//2]
        eval_indices = unsampled_indices[total_unsampled//2:]
        
        # Store the posterior in a num_nodes-by-num_classes matrix.
        # Posteriors in non-leaf cells will be zero everywhere
        # and later changed to uniform.
        node_counts = tree.tree_.n_node_samples
        class_counts = np.zeros((len(node_counts), K))
        est_nodes = tree.apply(X[vote_indices])
        est_classes = y[vote_indices]
        for i in range(len(est_nodes)):
            class_counts[est_nodes[i], est_classes[i]] += 1
        
        row_sums = class_counts.sum(axis=1) # Total number of estimation points in each leaf.
        row_sums[row_sums == 0] = 1 # Avoid divide by zero.
        class_probs = class_counts / row_sums[:, None]
        
        # Make the nodes that have no estimation indices uniform.
        # This includes non-leaf nodes, but that will not affect the estimate.
        class_probs[np.argwhere(class_probs.sum(axis = 1) == 0)] = [1 / K]*K
        
        # Apply finite sample correction and renormalize.
        where_0 = np.argwhere(class_probs == 0)
        for elem in where_0:
            class_probs[elem[0], elem[1]] = 1 / (kappa*class_counts.sum(axis = 1)[elem[0]])
        row_sums = class_probs.sum(axis=1)
        class_probs = class_probs / row_sums[:, None]
        
        # Place evaluation points in their corresponding leaf node.
        # Store evaluation posterior in a num_eval-by-num_class matrix.
        eval_class_probs = class_probs[tree.apply(X[eval_indices])]
        # eval_class_probs = [class_probs[x] for x in tree.apply(X[eval_indices])]
        eval_entropies = [entropy(posterior) for posterior in eval_class_probs]
        cond_entropy += np.mean(eval_entropies)
      
    return cond_entropy / n_estimators

def entropy_estimate(y, base = np.exp(1)):
    _, counts = np.unique(y, return_counts=True)
    return entropy(counts, base=base)

def estimate_mi(X, y):
    H_Y = entropy_estimate(y)
    H_YX = uf(X, y)
    return H_Y - H_YX

In [4]:
def _perm_stat(calc_stat, x, y):
    permy = np.random.permutation(y)
    perm_stat = calc_stat(x, permy)

    return perm_stat

def perm_test(calc_stat, X, y, reps=1000, workers=1):
    """
    Calculate the p-value via permutation
    """
    # calculate observed test statistic
    stat = calc_stat(X, y)

    # calculate null distribution
    null_dist = np.array(
        Parallel(n_jobs=workers)(
            [delayed(_perm_stat)(calc_stat, X, y) for rep in range(reps)]
        )
    )
    pvalue = (null_dist >= stat).sum() / reps

    # correct for a p-value of 0. This is because, with bootstrapping
    # permutations, a p-value of 0 is incorrect
    if pvalue == 0:
        pvalue = 1 / reps

    return stat, pvalue

In [5]:
reps = 1000

stat, pvalue = perm_test(estimate_mi, X, y, reps=reps, workers=-2)
print("Test Statistic: ", stat)
print("p-value: ", pvalue)

pickle.dump((stat, pvalue), open('stat_pval.pkl', 'wb'))

Test Statistic:  0.8430897838865855
p-value:  0.001


In [6]:
stat, pvalue = pickle.load(open('stat_pval.pkl', 'rb'))
print("Test Statistic: ", stat)
print("p-value: ", pvalue)

Test Statistic:  0.8430897838865855
p-value:  0.001
